In [21]:
import pandas as pd
import numpy as np

file_path = "../data/stooq/us/A.csv"
df = pd.read_csv(file_path)

# 수익률 계산: 종가 기준으로 % 계산
df['Return'] = df['Close'].pct_change() * 100
df = df.dropna(subset=['Return'])
df.head()

,Date,Open,High,Low,Close,Volume,ticker,Return
1,1999-11-19,27.8972,27.9371,25.8613,26.2311,1.677358e+07,A,-8.237307
2,1999-11-22,26.8370,28.5858,26.0278,28.5858,7.242576e+06,A,8.976749
3,1999-11-23,27.6102,28.3377,25.9889,25.9889,6.579458e+06,A,-9.084580
4,1999-11-24,26.0637,27.2445,25.9889,26.6745,5.332648e+06,A,2.638049
5,1999-11-26,26.5569,26.9585,26.4752,26.7622,1.904229e+06,A,0.328778


In [22]:
from scipy.stats.mstats import winsorize

# Winsorization 적용
df['Return_winsorized'] = winsorize(df['Return'], limits=[0.01, 0.01])
df[['Date', 'Return', 'Return_winsorized']].head(10)

,Date,Return,Return_winsorized
1,1999-11-19,-8.237307,-7.175549
2,1999-11-22,8.976749,7.946328
3,1999-11-23,-9.084580,-7.175549
4,1999-11-24,2.638049,2.638049
5,1999-11-26,0.328778,0.328778
6,1999-11-29,2.244957,2.244957
7,1999-11-30,0.175419,0.175419
8,1999-12-01,1.773740,1.773740
9,1999-12-02,2.754040,2.754040
10,1999-12-03,0.848407,0.848407


In [23]:
# 최대,최소 몇 % 인지 (논문에서 보통 -100% 안넘는다 함)
print(df['Return'].min(), df['Return'].max())

-27.08045384540241 47.18068071848707


In [24]:
import pandas as pd
import math

file_path = "../data/stooq/us/A.csv"
df = pd.read_csv(file_path)

# 수익률 계산
df['Return'] = df['Close'].pct_change()

def discretize_return(r):
    """
    단일 수익률 r (소수 형태, 예: -0.024는 -2.4%)를 토큰 인덱스 (0 ~ 401)로 변환
    
    변환 규칙:
      - r를 10,000배하여 basis point 정수값으로 변환 (r_bp)
      - r_bp <= -10000  → 토큰 0
      - r_bp > 10000    → 토큰 401
      - 그 외: 토큰 = ceil((r_bp + 10000) / 50)
              (이때 토큰은 1부터 400까지 할당)
    """
    r_bp = int(r * 10000)
    if r_bp <= -10000:
        return 0
    elif r_bp > 10000:
        return 401
    else:
        return math.ceil((r_bp + 10000) / 50)

# Return 열의 각 값에 대해 토큰 인덱스 계산 (첫 행은 NaN이므로 처리)
df = df.dropna(subset=['Return'])
df['ReturnToken'] = df['Return'].apply(lambda x: discretize_return(x))

output_file_path = "../data/model/A_with_returns_tokens.csv"
df.to_csv(output_file_path, index=False)

print(df.head(10))


          Date     Open     High      Low    Close        Volume ticker  \
1   1999-11-19  27.8972  27.9371  25.8613  26.2311  1.677358e+07      A   
2   1999-11-22  26.8370  28.5858  26.0278  28.5858  7.242576e+06      A   
3   1999-11-23  27.6102  28.3377  25.9889  25.9889  6.579458e+06      A   
4   1999-11-24  26.0637  27.2445  25.9889  26.6745  5.332648e+06      A   
5   1999-11-26  26.5569  26.9585  26.4752  26.7622  1.904229e+06      A   
6   1999-11-29  26.6356  27.5724  26.3506  27.3630  4.486512e+06      A   
7   1999-11-30  27.2844  27.8972  26.5958  27.4110  4.745571e+06      A   
8   1999-12-01  27.4110  28.2221  27.2096  27.8972  3.256172e+06      A   
9   1999-12-02  28.4204  29.2325  28.0596  28.6655  3.380083e+06      A   
10  1999-12-03  29.1957  29.6819  28.7851  28.9087  3.348990e+06      A   

      Return  ReturnToken  
1  -0.082373          184  
2   0.089767          218  
3  -0.090846          182  
4   0.026380          206  
5   0.003288          201  
6   0.

In [25]:
def decode_return(token):
    """
    단일 토큰 인덱스(token: 0 ~ 401)를 해당하는 수익률(소수 형태)로 디코딩합니다.
    
    변환 규칙:
      - token == 0   -> -10000 basis points (-100%)
      - token == 401 -> +10000 basis points (+100%)
      - token 1 ~ 400: 해당 구간의 대표값(중간값)은
                       -10000 + (token - 1) * 50 + 25
                       basis point 단위이며, 이를 10000으로 나누어 소수 형태로 반환합니다.
    """
    if token == 0:
        r_bp = -10000
    elif token == 401:
        r_bp = 10000
    else:
        r_bp = -10000 + (token - 1) * 50 + 25
    return r_bp / 100.0

def decode(token_list):
    """
    token_list: 정수 토큰의 리스트 (예: [196, 200, 200, 210, 210])
    
    각 토큰을 decode_return 함수를 사용해 디코딩하고,
    디코딩된 수익률의 리스트를 반환합니다.
    """
    return [decode_return(token) for token in token_list]

tokens = [196, 200, 200, 210, 210]
print("Decoded returns:", decode(tokens))

Decoded returns: [-2.25, -0.25, -0.25, 4.75, 4.75]


In [26]:
# 논문에 나온 예시 테스트
sample = [-0.024, 0, 0, 0.05, 0.048]
sample = [discretize_return(x) for x in sample]
sample

[196, 200, 200, 210, 210]

In [27]:
import os
import glob

def process_csv_file(file_path, output_dir):
    """
    파일을 읽어서 일별 수익률과 토큰 인덱스를 계산한 후, 결과를 output_dir에 저장합니다.
    """
    # CSV 파일 로드
    try:
        df = pd.read_csv(file_path)
    except pd.errors.EmptyDataError:
        print(f"Skipping file {file_path}: File is empty.")
        return

    # 'Close' 혹은 'close' 열 확인
    if 'Close' in df.columns:
        close_col = 'Close'
    else:
        print(f"Skipping file {file_path} because it does not have a 'Close' column.")
        return  # 해당 파일은 처리하지 않고 건너뜁니다.
    
    # Close 기준 일별 수익률 계산: (오늘의 Close / 어제의 Close) - 1
    df['Return'] = df[close_col].pct_change()
    
    df = df.dropna(subset=['Return'])
    df['ReturnToken'] = df['Return'].apply(lambda x: discretize_return(x))
    
    # 원본 파일 이름에 접미사를 붙여 결과 파일 이름 생성
    base_name = os.path.basename(file_path)
    output_file = os.path.join(output_dir, base_name.replace('.csv', '_with_returns_tokens.csv'))
    
    # 결과 DataFrame을 CSV 파일로 저장 (index는 제외)
    df.to_csv(output_file, index=False)
    print(f"Processed: {file_path} -> {output_file}")

In [28]:
input_dir = "../data/stooq/us"
output_dir = "../data/model"

# 출력 폴더가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 입력 폴더 내의 모든 CSV 파일 목록을 가져옴
csv_files = glob.glob(os.path.join(input_dir, "*.csv"))

# 각 파일에 대해 처리 수행
for file_path in csv_files:
    process_csv_file(file_path, output_dir)

Processed: ../data/stooq/us/ANTE.csv -> ../data/model/ANTE_with_returns_tokens.csv
Processed: ../data/stooq/us/AIRT.csv -> ../data/model/AIRT_with_returns_tokens.csv
Processed: ../data/stooq/us/AISP.csv -> ../data/model/AISP_with_returns_tokens.csv
Processed: ../data/stooq/us/ALCY.csv -> ../data/model/ALCY_with_returns_tokens.csv
Processed: ../data/stooq/us/AEIS.csv -> ../data/model/AEIS_with_returns_tokens.csv
Processed: ../data/stooq/us/ALL_I.csv -> ../data/model/ALL_I_with_returns_tokens.csv
Processed: ../data/stooq/us/ALTG.csv -> ../data/model/ALTG_with_returns_tokens.csv
Processed: ../data/stooq/us/ALVO.csv -> ../data/model/ALVO_with_returns_tokens.csv
Processed: ../data/stooq/us/AIFF.csv -> ../data/model/AIFF_with_returns_tokens.csv
Processed: ../data/stooq/us/AMSC.csv -> ../data/model/AMSC_with_returns_tokens.csv
Processed: ../data/stooq/us/AMPX.csv -> ../data/model/AMPX_with_returns_tokens.csv
Processed: ../data/stooq/us/AISPW.csv -> ../data/model/AISPW_with_returns_tokens.csv


In [31]:
import os
import glob

data_dir = "../data/model"
files = glob.glob(os.path.join(data_dir, "*"))
print(f"'{data_dir}' 폴더 내 파일 개수: {len(files)}")

'../data/model' 폴더 내 파일 개수: 520


In [32]:
import torch
import pandas as pd
import random

torch.manual_seed(1337)
batch_size = 64    
block_size =256    

def load_return_tokens_from_file(file_path):
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None
    
    if 'ReturnToken' not in df.columns:
        print(f"File {file_path} does not have 'ReturnToken' column. Skipping.")
        return None

    tokens = df['ReturnToken'].dropna().tolist()
    return torch.tensor(tokens, dtype=torch.long)

In [33]:
data_dir = "../data/model"
csv_files = glob.glob(os.path.join(data_dir, "*_with_returns_tokens.csv"))

print(len(csv_files))

train_data_list = []
for file_path in csv_files:
    tokens_tensor = load_return_tokens_from_file(file_path)
    if tokens_tensor is not None and len(tokens_tensor) > block_size: # block size 보다 길이 작은 파일들은 제외함
        train_data_list.append(tokens_tensor)

train_data_list[:5]
print(f"총 {len(train_data_list)}개의 파일에서 ReturnToken 시퀀스를 로드했습니다.")

520
총 448개의 파일에서 ReturnToken 시퀀스를 로드했습니다.


In [34]:
# train_data_list를 무작위로 섞은 후, train/val split (예: 80% train, 20% val)
train_val_ratio = 0.8
random.shuffle(train_data_list)
n_train = int(len(train_data_list) * train_val_ratio)
train_list = train_data_list[:n_train]
val_list = train_data_list[n_train:]

print(f"Train 파일 수: {len(train_list)}, Val 파일 수: {len(val_list)}")

Train 파일 수: 358, Val 파일 수: 90


In [35]:
def get_batch(split='train'):
    """
    각 배치 샘플은 한 파일 내에서 block_size 길이의 입력(x)와 그 바로 다음 토큰(y)로 구성합니다.
    파일 간의 토큰이 섞이지 않습니다.
    """
    if split == 'train':
        data_list = train_list
    else:
        data_list = val_list
    
    xs, ys = [], []
    for _ in range(batch_size):
        # block_size+1 이상의 길이를 가진 파일 중에서 랜덤 선택
        valid_files = [d for d in data_list if len(d) > block_size]

        if not valid_files:
            raise ValueError("block_size보다 긴 ReturnToken 시퀀스가 있는 파일이 없습니다.")

        data = random.choice(valid_files)
        start_idx = random.randint(0, len(data) - block_size - 1)
        x = data[start_idx : start_idx + block_size]
        y = data[start_idx + 1 : start_idx + block_size + 1]
        xs.append(x)
        ys.append(y)
    
    # 배치 차원으로 스택
    x_batch = torch.stack(xs)  # (batch_size, block_size)
    y_batch = torch.stack(ys)  # (batch_size, block_size)
    return x_batch, y_batch

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)  
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')
for b in range(5):  # batch 차원
    for t in range(block_size):  # 시간 차원
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([64, 256])
tensor([[197, 233, 178,  ..., 194, 194, 200],
        [201, 200, 204,  ..., 197, 204, 200],
        [204, 207, 192,  ..., 197, 198, 184],
        ...,
        [202, 203, 199,  ..., 198, 203, 196],
        [214, 200, 201,  ..., 197, 196, 196],
        [212, 208, 208,  ..., 199, 205, 221]])
targets:
torch.Size([64, 256])
tensor([[233, 178, 185,  ..., 194, 200, 214],
        [200, 204, 194,  ..., 204, 200, 203],
        [207, 192, 193,  ..., 198, 184, 201],
        ...,
        [203, 199, 197,  ..., 203, 196, 200],
        [200, 201, 197,  ..., 196, 196, 182],
        [208, 208, 193,  ..., 205, 221, 207]])
----
when input is [197] the target: 233
when input is [197, 233] the target: 178
when input is [197, 233, 178] the target: 185
when input is [197, 233, 178, 185] the target: 226
when input is [197, 233, 178, 185, 226] the target: 178
when input is [197, 233, 178, 185, 226, 178] the target: 214
when input is [197, 233, 178, 185, 226, 178, 214] the target: 1